In [1]:
import os
import torch
import random
import numpy as np
import pandas as pd
from datasets import load_dataset
import nlpaug.augmenter.sentence as nas
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split
from transformers import MarianMTModel, MarianTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler

/opt/conda/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
aug = nas.LambadaAug(model_dir='./Model', device=device)

FileNotFoundError: [Errno 2] No such file or directory: './Model/cls/label_encoder.json'

In [ ]:
def get_dataset(dataset_name):
    if dataset_name == 'covid':
        # Load the dataset into a pandas dataframe.
        dataset = load_dataset('llangnickel/long-covid-classification-data')
        # get the training data
        batch = dataset['train']['text']
        labels = dataset['train']['label']
        
    elif dataset_name == 'cancer':
        classes = {'Thyroid_Cancer' : 0,  'Lung_Cancer' : 1,  'Colon_Cancer' : 2}
        # Load the dataset into a pandas dataframe.
        df = pd.read_csv('../../cancer.csv', encoding='latin')
        values = df.values

        # get the entire data
        all_batch  = list(values[:,2])
        str_labels = list(values[:,1])
        all_labels = [classes[k] for k in str_labels]

        # training and test data split
        batch, _, labels, _ = train_test_split(all_batch, all_labels, test_size=0.5, random_state=42)
        batch = batch[:100] # since it is an easy dataset use just 500 data points
        labels = labels[:100]
        
    return batch, labels

In [ ]:
dataset = 'covid'
original_texts, original_labels = get_dataset(dataset)

In [ ]:
truncated_texts = []
for i in range(len(original_texts)):
    truncated_texts.append(' '.join(original_texts[i].split()[:256]))

In [ ]:
augmented_data = []

for i in range(len(truncated_texts)):
    augmented_data.append(aug.augment(truncated_texts[i]))

In [ ]:
f = open("Data/abstractivesummarization_"+str(dataset)+".txt" , "a" )

for i in range(len(augmented_data)):
    f.write(str(original_labels[i]) + '\t' + augmented_data[i][0] + '\n')
    
f.close()

In [ ]:
augmented_data[10][0]

In [ ]:
original_labels[10]